In [1]:
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import GlobalAveragePooling2D, Flatten, Dense, Input, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2


file = 'all'
if file=='all':
    path = '/home/ubuntu/nbs/state_farm/data/'
    batch_size=64
    nb_train_samples = 34000
    nb_validation_samples = 4800
elif file=='sample':
    path = '/home/ubuntu/nbs/state_farm/data/sample/'
    batch_size=64
    nb_train_samples = 1600
    nb_validation_samples = 300
elif file=='mini_sample':
    path = '/home/ubuntu/nbs/state_farm/data/mini_sample/'
    batch_size=1
    nb_train_samples = 200
    nb_validation_samples = 20
else:
    print('Unknown file name')
    exit()

#path = '/home/ubuntu/nbs/state_farm/data/'
target_size=(224, 224)

#generate the batches
def get_batches(directory, target_size=target_size, batch_size=batch_size, shuffle=False):
    datagen = ImageDataGenerator()
    return datagen.flow_from_directory(directory=directory,
                                          target_size=target_size,
                                          batch_size=batch_size,
                                          class_mode='categorical',
                                          shuffle=shuffle)

train_gen = ImageDataGenerator(rotation_range=0, 
                                     width_shift_range=0, 
                                     height_shift_range=0.1, 
                                     shear_range=0.1, 
                                     zoom_range=0.2, 
                                     channel_shift_range=10,
                                     fill_mode='nearest')

batches = train_gen.flow_from_directory(directory=path+'train_set',  
                                        target_size=target_size,
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        shuffle=True)

#batches = get_batches(path+'train_set', shuffle=True)

valid_batches = get_batches(path+'valid_set', batch_size=batch_size*2, shuffle=False)



Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)


Found 17616 images belonging to 10 classes.
Found 4808 images belonging to 10 classes.


In [2]:
# initialize the model
initial_model = VGG16(weights="imagenet", include_top = False, input_shape = (224,224,3))


In [3]:
x = Flatten()(initial_model.output)
x = Dense(4096, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(batches.num_class, activation = 'softmax')(x)
model = Model(input = initial_model.input, output = predictions)


# we freeze the other layers 
for layer in initial_model.layers: layer.trainable=False

opt = Adam(lr=10e-5)

model.compile(optimizer=opt,
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.fit_generator(batches, epochs=10, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=valid_batches, 
                    validation_steps=nb_validation_samples // batch_size)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Softmax.0, inputs=/input_1)`


Epoch 1/10
531/531 [==============================] - 968s - loss: 0.1984 - acc: 0.9412 - val_loss: 0.9099 - val_acc: 0.7347
Epoch 2/10
531/531 [==============================] - 971s - loss: 0.0434 - acc: 0.9883 - val_loss: 0.9413 - val_acc: 0.7664
Epoch 3/10
531/531 [==============================] - 970s - loss: 0.0249 - acc: 0.9941 - val_loss: 0.8107 - val_acc: 0.7852
Epoch 4/10
531/531 [==============================] - 970s - loss: 0.0219 - acc: 0.9944 - val_loss: 0.9418 - val_acc: 0.7602
Epoch 5/10
531/531 [==============================] - 969s - loss: 0.0156 - acc: 0.9962 - val_loss: 0.9501 - val_acc: 0.7577
Epoch 6/10
531/531 [==============================] - 969s - loss: 0.0138 - acc: 0.9966 - val_loss: 0.9607 - val_acc: 0.7734
Epoch 7/10
531/531 [==============================] - 968s - loss: 0.0115 - acc: 0.9971 - val_loss: 0.9675 - val_acc: 0.7557
Epoch 8/10
531/531 [==============================] - 970s - loss: 0.0123 - acc: 0.9970 - val_loss: 0.8961 - val_acc: 0.7383


In [4]:
#model.save('models/batchnorm_step1.h5')

for layer in model.layers[:10]: 
    layer.trainable = False 

for layer in model.layers[10:]: 
    layer.trainable = True
    
opt = SGD(lr=10e-5) 
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(batches, epochs=10, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_data=valid_batches, 
                    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
531/531 [==============================] - 1531s - loss: 0.0106 - acc: 0.9978 - val_loss: 0.9060 - val_acc: 0.7525
Epoch 2/10
531/531 [==============================] - 1531s - loss: 0.0089 - acc: 0.9979 - val_loss: 0.8796 - val_acc: 0.7583
Epoch 3/10
531/531 [==============================] - 1531s - loss: 0.0072 - acc: 0.9986 - val_loss: 0.8674 - val_acc: 0.7589
Epoch 4/10
531/531 [==============================] - 1533s - loss: 0.0064 - acc: 0.9990 - val_loss: 0.8659 - val_acc: 0.7638
Epoch 5/10
531/531 [==============================] - 1528s - loss: 0.0063 - acc: 0.9986 - val_loss: 0.8601 - val_acc: 0.7673
Epoch 6/10
531/531 [==============================] - 1532s - loss: 0.0058 - acc: 0.9988 - val_loss: 0.8766 - val_acc: 0.7653
Epoch 7/10
531/531 [==============================] - 1532s - loss: 0.0064 - acc: 0.9985 - val_loss: 0.8822 - val_acc: 0.7649
Epoch 8/10
531/531 [==============================] - 1533s - loss: 0.0058 - acc: 0.9986 - val_loss: 0.8756 - val_acc:

In [31]:
# predictions on new data
print('start!')
model.save('models/batchnorm_ft.h5')
test_batches = get_batches(path+'test', batch_size=batch_size)
preds = model.predict_generator(test_batches, steps=test_batches.samples)
preds = preds.clip(min=0.02, max=0.98)

start!
Found 79726 images belonging to 1 classes.


KeyboardInterrupt: 

In [32]:
import numpy as np
import pandas as pd
filenames = test_batches.filenames
ids = np.array([f[8:] for f in filenames])

df_subm = pd.DataFrame(preds, index=ids, columns=['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])
df_subm.reset_index(level=0, inplace=True)
df_subm=df_subm.rename(columns = {'index':'img'})
df_subm.head()

In [ ]:
from IPython.display import FileLink
submission_file_name = 'submission.csv.gz'
df_subm.to_csv(submission_file_name, index=False, compression='gzip')
FileLink(submission_file_name)